Import libraies

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time

from sklearn.metrics import mean_absolute_error


Read train and test data files

In [2]:
def read_file(file):
	File = open(file, "r")
	data = {"id": [], "corpus": [],  "sentence": [],"token" :[], "complexity": []}
	
	File.readline()
	while True:
		line = File.readline()
		if len(line) == 0:
			break

		line = line.split()
		data["id"].append(line[0])
		data["corpus"].append(line[1])

		data["sentence"].append(' '.join(line[2:-2]))
		data["token"].append(line[-2])
		data["complexity"].append(float(line[-1]))
	return pd.DataFrame(data)


In [3]:
test = read_file('test.txt' )

In [4]:
test.head()

,id,corpus,sentence,token,complexity
0,3K8CQCU3KE19US5SN890DFPK3SANWR,bible,"But he, beckoning to them with his hand to be ...",hand,0.000000
1,3Q2T3FD0ON86LCI41NJYV3PN0BW3MV,bible,"If I forget you, Jerusalem, let my right hand ...",hand,0.197368
2,3ULIZ0H1VA5C32JJMKOTQ8Z4GUS51B,bible,"the ten sons of Haman the son of Hammedatha, t...",hand,0.200000
3,3BFF0DJK8XCEIOT30ZLBPPSRMZQTSD,bible,Let your hand be lifted up above your adversar...,hand,0.267857
4,3QREJ3J433XSBS8QMHAICCR0BQ1LKR,bible,"Abimelech chased him, and he fled before him, ...",entrance,0.000000


Data cleaning and Preprocessing Data

In [5]:
import nltk
import string
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [6]:
lemmatizer = WordNetLemmatizer() 
def lemmatize(text):
	lemmas = [lemmatizer.lemmatize(word) for word in word_tokenize(text)] 
	return " ".join(lemmas)

# remove stopwords
def remove_stopwords(text, token):
	words = [w for w in word_tokenize(text) if (w not in stopwords.words('english')) or w in list(word_tokenize(token))]
	return " ".join(words)

In [7]:
def preprocess(data):
	
	data["token"] = data['token'].astype(str)

	# lower case
	data['sentence'] = data['sentence'].apply(lambda x: x.lower())
	data['token'] = data['token'].apply(lambda x: x.lower())

	# remove punctuation
	translator = str.maketrans('', '', string.punctuation)
	data['sentence'] = data['sentence'].apply(lambda x: x.translate(translator))
	data['token'] = data['token'].apply(lambda x: x.translate(translator))

	# remove stopwords only if token != a stopword
	try:
		nltk.data.find('corpora/stopwords')
	except LookupError:
		nltk.download('stopwords')

	data['sentence'] = data.apply(lambda x: remove_stopwords(x['sentence'], x['token']) , axis=1)
	
	# lemmatize
	data['sentence'] = data['sentence'].apply(lambda x: lemmatize(x))
	data['token'] = data['token'].apply(lambda x: lemmatize(x))

	return data

In [8]:
test_data = preprocess(test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
test_data.head()

,id,corpus,sentence,token,complexity
0,3K8CQCU3KE19US5SN890DFPK3SANWR,bible,beckoning hand silent declared lord brought pr...,hand,0.000000
1,3Q2T3FD0ON86LCI41NJYV3PN0BW3MV,bible,forget jerusalem let right hand forget skill,hand,0.197368
2,3ULIZ0H1VA5C32JJMKOTQ8Z4GUS51B,bible,ten son haman son hammedatha jew enemy didnt l...,hand,0.200000
3,3BFF0DJK8XCEIOT30ZLBPPSRMZQTSD,bible,let hand lifted adversary let enemy cut,hand,0.267857
4,3QREJ3J433XSBS8QMHAICCR0BQ1LKR,bible,abimelech chased fled many fell wounded even e...,entrance,0.000000


Creating new Features for data

In [10]:
test_data['token_length'] = test_data['token'].str.len()
test_data.head()

,id,corpus,sentence,token,complexity,token_length
0,3K8CQCU3KE19US5SN890DFPK3SANWR,bible,beckoning hand silent declared lord brought pr...,hand,0.000000,4
1,3Q2T3FD0ON86LCI41NJYV3PN0BW3MV,bible,forget jerusalem let right hand forget skill,hand,0.197368,4
2,3ULIZ0H1VA5C32JJMKOTQ8Z4GUS51B,bible,ten son haman son hammedatha jew enemy didnt l...,hand,0.200000,4
3,3BFF0DJK8XCEIOT30ZLBPPSRMZQTSD,bible,let hand lifted adversary let enemy cut,hand,0.267857,4
4,3QREJ3J433XSBS8QMHAICCR0BQ1LKR,bible,abimelech chased fled many fell wounded even e...,entrance,0.000000,8


In [11]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [12]:
test_data['syllables'] = test_data['token'].apply(lambda x: syllable_count(x) )

In [13]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d embeddings
!rm glove.6B.zip
!rm embeddings/glove.6B.50d.txt
!rm embeddings/glove.6B.100d.txt
!rm embeddings/glove.6B.200d.txt

--2022-12-21 18:05:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-12-21 18:05:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-12-21 18:05:35--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [14]:

test_data['token'] = test_data['token'].astype(str)
test_data['sentence'] = test_data['sentence'].astype(str)

In [15]:

test_sentence_list = list(test['sentence'])
test_complexity_list = list(test['complexity'])
test_token_list = list(test['token'])


Create Embedding from Glove Embeddings

In [16]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec_map

word_to_vec_map = read_glove_vector('embeddings/glove.6B.300d.txt')

# get embeddings, and pad till max_len
def get_embeddings(sentences, max_len=0):
    sentence_emb = []
    for s in sentences:

        temp_sent_emb = [ word_to_vec_map[x] if x in word_to_vec_map else np.full((300,), np.nan) for x in s.split() ]
        mean_emb = np.nanmean(np.array(temp_sent_emb), axis=0)

        temp_sent_emb = np.array([ mean_emb if np.isnan(x[0]) else x for x in temp_sent_emb ])
        temp_sent_emb = np.concatenate((temp_sent_emb, np.zeros((max_len-temp_sent_emb.shape[0],300))))

        sentence_emb.append(temp_sent_emb)

    return np.array(sentence_emb)

Create Padding for sentences

In [17]:

# max len for the sentence used for padding
max_len_sent = max([len(s.split()) for s in test_sentence_list])

print(max_len_sent)



115


In [18]:
test_sent_emb = get_embeddings(test_sentence_list, max_len_sent)

print("Sentence embedding shape test : {}".format(test_sent_emb.shape))


Sentence embedding shape test : (917, 115, 300)


Create Positional data values from token : Location of token in senetence

In [19]:


test_token_data =   [ 
                        [ 
                            len(s.split(t)[0].split()), 
                            len(t.split())
                        ]
                        for s,t in zip(test_sentence_list, test_token_list) 
                    ]


In [20]:


features_test = test[['token_length', 'syllables',
                        ]].values


In [21]:
class FFN(tf.keras.layers.Layer):
    def __init__(
        self, 
        hidden_size,
        output_size,
        rate,
        softmax=False,
    ):
        super(FFN, self).__init__()

        self.layer1 = tf.keras.layers.Dense(hidden_size, activation="relu")  # (batch_size, hidden_size)
        self.dropout = tf.keras.layers.Dropout(rate)
        self.layer2 = tf.keras.layers.Dense(output_size, activation= "softmax" if softmax is True else None)  # (batch_size, output_size)

    def call(self, x, training):
        return self.layer2(self.dropout(self.layer1(x), training=training) ) 
        # return self.layer2(self.dropout(x, training=training)) 


In [22]:
class OurModelBiLSTM(tf.keras.Model):
    def __init__(
        self,
        lstm_units,
        hidden_size,
        random_seed,
        seq_len,
        embedding_size,
        rate=0.25
        ):
        """
        hidden_size - for FFN
        """

        super(OurModelBiLSTM, self).__init__()

        tf.random.set_seed(random_seed)
        self.random_seed = random_seed
        self.lstm_units = lstm_units

        self.bilstm = tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(self.lstm_units, return_sequences=True), input_shape=(seq_len,embedding_size)
            )
        self.dropout = tf.keras.layers.Dropout(rate=rate)

        self.layernorm = tf.keras.layers.LayerNormalization()
        self.dense = tf.keras.layers.Dense(2*self.lstm_units)
        self.getWeights = FFN(hidden_size, 3, rate, softmax=True)
        self.final_layer = FFN(hidden_size, 1, rate)

    
    # one fwd pass on the model
    def call(self, input_seq, token_position, feature_emb, training=False):
        """
        input_seq : [batch_size, seq_len, embedding_size]
        token_position = [batch_size, 2] - [start,length] for tokens for which complexity is to be predicted
        """

        # bi-lstm pass
        bilstm_output = self.bilstm(input_seq)

        # bilstm_output = self.bilstm(input_seq)      #(batch_size, seq_len, 2*lstm_units)
        bilstm_output = self.dropout(bilstm_output, training=training)

        # extract token embeddings    
        # for each example we have start token and length
        # we take mean of these embeddings position

        token_emb = tf.stack(
                [tf.reduce_mean(tf.gather(i, tf.range(j[0],j[0]+j[1]), axis=0), axis=0) for i,j in zip(bilstm_output,token_position) ]
            )   # (batch_size, 2*lstm_units)

        # combine all embeddings - take mean
        added_emb = tf.reduce_mean(bilstm_output, axis=1)    # (batch_size, 2*lstm_units)
        
        # feature_emb - (batch_shape, features)
        feature_emb = self.dense(feature_emb)           # (batch_size, 2*lstm_units)
        # feature_emb = tf.cast(feature_emb, tf.float32)

        # get weights
        weights = self.getWeights(self.layernorm(tf.concat([token_emb,added_emb, feature_emb], axis=1)), training)    # (batch_size, 3)

        # generate attenton-based final embeddings
        # final_emb = weights[0]*token_emb + weights[1]*added_emb + weights[3]*features_emb

        final_emb = tf.zeros(shape=(), dtype=tf.dtypes.float32)   # (bacth_size, 2*lstm_units)
        final_emb += tf.expand_dims(weights[:, 0], axis=1) * token_emb
        final_emb += tf.expand_dims(weights[:, 1], axis=1) * added_emb
        final_emb += tf.expand_dims(weights[:, 2], axis=1) * feature_emb

        # output complexity
        final_output = self.final_layer(final_emb, training)

        return final_output

    # loss function
    def loss_function(self, real, pred):
        loss_ = tf.keras.losses.MSE(real, pred)
        l = tf.reduce_mean(loss_)
        return l

    # set optimizer
    def compile(self, optimizer):
        super(OurModelBiLSTM, self).compile()
        self.optimizer = optimizer


    # train step - fwd pass + back prop + update model weights
    # @tf.function() - may not work beacuse zip is used
    def train_step(self, input_seq, token_position, y, feature_emb):
        training=True
        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self(input_seq, token_position, feature_emb, training)  
            loss = self.loss_function(tf.reshape(y,(-1,1)), y_pred)

        # calculate gradients
        gradients = tape.gradient(loss, self.trainable_variables)

        # update model weights using gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return loss

    # fit funtion of model
    def _fit(
        self, 
        input_seq_data, 
        token_position_data, 
        feature_emb_data,
        complexity, 
        epochs, 
        batch_size,
        val_input_seq_data=None,
        val_token_position_data=None,
        val_feature_emb_data=None,
        val_complexity=None
        ):

        # create tensorflow dataset
        tf_data = tf.data.Dataset.from_tensor_slices((input_seq_data, token_position_data, feature_emb_data, complexity))

        # shuffle and batch
        tf_data = tf_data.shuffle(100000, seed=self.random_seed).batch(batch_size)

        train_loss = tf.keras.metrics.Mean(name="train_loss")
        loss_train = {"MSE": []}
        loss_val = {"MSE": []}

        #training starts
        for epoch in range(epochs):
            start = time.time()
            
            # reset state for mean loss
            train_loss.reset_states()

            # loop over batches
            for step, x in tf_data.enumerate():

                # get inputs for model
                input_seq = x[0]
                token_position = x[1]
                feature_emb = x[2]
                y = x[3]

                # call trainstep
                loss = self.train_step(input_seq, token_position, y, feature_emb)
                train_loss(loss)

            loss_train["MSE"].append(train_loss.result().numpy())
            print("Epoch {} loss  MSE: {}, time taken: {:.2f}s".format(epoch + 1, loss_train["MSE"][-1], time.time() - start))

            # validation if provided
            if (val_input_seq_data is not None):
                val_pred = self._predict(val_input_seq_data, val_token_position_data, val_feature_emb_data)
                loss_val["MSE"].append(self.loss_function(tf.reshape(val_complexity, (-1,1)), val_pred).numpy())
                print("Validation loss MSE : {}".format(loss_val["MSE"][-1]))


        if (val_input_seq_data is not None):
            return loss_train, loss_val
        return loss_train

    
    # predict function
    def _predict(self, input_seq_data, token_position_data, feature_emb_data):
        
        # create tensorflow dataset
        tf_data = tf.data.Dataset.from_tensor_slices((input_seq_data, token_position_data, feature_emb_data))

        # batch for memory constraints
        tf_data = tf_data.batch(512)

        pred_list = []
        for step, x in tf_data.enumerate():
            # get inputs for model
            input_seq = x[0]
            token_position = x[1]
            feature_emb = x[2]

            pred_list.append(self(input_seq, token_position, feature_emb))
        return tf.concat(pred_list, axis=0)
     

In [23]:
lstm_units=20
hidden_size=10
random_seed=12
seq_len=max_len_sent
embedding_size=300
rate=0.4
epochs=20
batch_size=32
model2 = OurModelBiLSTM(lstm_units=lstm_units, hidden_size=hidden_size, random_seed=random_seed, seq_len=seq_len, embedding_size=embedding_size, rate=rate)
model2.load_weights('./my_modal')
pred2 = model2._predict(test_sent_emb, test_token_data, features_test)
mae_score = mean_absolute_error(test_complexity_list, pred2)
print(mae_score)
     

0.07109030243745054
